In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
data_dir = "/content/drive/MyDrive/archive 2/real_and_fake_face"


In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import classification_report, confusion_matrix


In [ ]:
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input

img_size = (128, 128)
batch_size = 32

train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,  # <-- Important change for transfer learning
    validation_split=0.2,
    horizontal_flip=True,
    zoom_range=0.2,
    shear_range=0.2,
    rotation_range=20
)

train_generator = train_datagen.flow_from_directory(
    data_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='binary',
    subset='training',
    shuffle=True
)

val_generator = train_datagen.flow_from_directory(
    data_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='binary',
    subset='validation',
    shuffle=True
)

Found 1649 images belonging to 2 classes.
Found 412 images belonging to 2 classes.


In [ ]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Input, GlobalAveragePooling2D, Dropout, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

# Define the input shape
img_shape = (128, 128, 3)

# 1. Load the pre-trained MobileNetV2 base
# We don't include the top classification layer and use weights from 'imagenet'
base_model = MobileNetV2(input_shape=img_shape,
                         include_top=False,
                         weights='imagenet')

# 2. Freeze the base model so we don't train it again
base_model.trainable = False

# 3. Add your own custom classification layers on top
inputs = Input(shape=img_shape)
# Pass the inputs to the base model
x = base_model(inputs, training=False)
# Add a pooling layer to reduce dimensions
x = GlobalAveragePooling2D()(x)
# Add a dropout layer for regularization
x = Dropout(0.2)(x)
# Add a dense layer for classification
x = Dense(128, activation='relu')(x)
# Add the final output layer
outputs = Dense(1, activation='sigmoid')(x)

# 4. Create the final model
model = Model(inputs, outputs)

# 5. Compile the model with a good starting learning rate
model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 128, 128, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ mobilenetv2_1.00_128            │ (None, 4, 4, 1280)     │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 1280)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │       163,968 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,422,081 (9.24 MB)

 Trainable params: 164,097 (641.00 KB)

 Non-trainable params: 2,257,984 (8.61 MB)

In [ ]:
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=50
)


Epoch 1/50
52/52 ━━━━━━━━━━━━━━━━━━━━ 862s 16s/step - accuracy: 0.5402 - loss: 0.8256 - val_accuracy: 0.4951 - val_loss: 0.7339
Epoch 2/50
52/52 ━━━━━━━━━━━━━━━━━━━━ 30s 588ms/step - accuracy: 0.6576 - loss: 0.6268 - val_accuracy: 0.5121 - val_loss: 0.8009
Epoch 3/50
52/52 ━━━━━━━━━━━━━━━━━━━━ 31s 591ms/step - accuracy: 0.7052 - loss: 0.5877 - val_accuracy: 0.5583 - val_loss: 0.7401
Epoch 4/50
52/52 ━━━━━━━━━━━━━━━━━━━━ 31s 596ms/step - accuracy: 0.6496 - loss: 0.6128 - val_accuracy: 0.5170 - val_loss: 0.7579
Epoch 5/50
52/52 ━━━━━━━━━━━━━━━━━━━━ 47s 713ms/step - accuracy: 0.7051 - loss: 0.5569 - val_accuracy: 0.5121 - val_loss: 0.7727
Epoch 6/50
52/52 ━━━━━━━━━━━━━━━━━━━━ 31s 597ms/step - accuracy: 0.7229 - loss: 0.5563 - val_accuracy: 0.5097 - val_loss: 0.8239
Epoch 7/50
52/52 ━━━━━━━━━━━━━━━━━━━━ 35s 686ms/step - accuracy: 0.7199 - loss: 0.5566 - val_accuracy: 0.5024 - val_loss: 0.9113
Epoch 8/50
52/52 ━━━━━━━━━━━━━━━━━━━━ 31s 601ms/step - accuracy: 0.7262 - loss: 0.5479 - val_accur

In [ ]:
# Evaluate model on validation/test set
loss, acc = model.evaluate(val_generator)
print(f"Validation Accuracy: {acc * 100:.2f}%")

# Generate predictions and true labels
val_generator.reset()
y_pred = model.predict(val_generator)
y_pred_labels = (y_pred > 0.5).astype(int).flatten()

y_true = val_generator.classes

# Class names from directory
class_labels = list(val_generator.class_indices.keys())

# Classification Report
from sklearn.metrics import classification_report
print(classification_report(y_true, y_pred_labels, target_names=class_labels))


13/13 ━━━━━━━━━━━━━━━━━━━━ 6s 479ms/step - accuracy: 0.5423 - loss: 1.1583
Validation Accuracy: 53.64%
13/13 ━━━━━━━━━━━━━━━━━━━━ 12s 680ms/step
               precision    recall  f1-score   support

training_fake       0.45      0.33      0.38       192
training_real       0.52      0.64      0.58       220

     accuracy                           0.50       412
    macro avg       0.49      0.49      0.48       412
 weighted avg       0.49      0.50      0.49       412



In [ ]:
# Import all necessary libraries for this cell
import os
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from google.colab import files

# --- 1. Define the Detailed Prediction Function ---
def predict_with_details(img_path):
    # Load and display the image
    img = load_img(img_path, target_size=img_size)
    plt.imshow(img)
    plt.title(f"Predicting: {os.path.basename(img_path)}")
    plt.axis('off') # Hide the axes
    plt.show()

    # Preprocess the image for the model
    img_array = img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = preprocess_input(img_array)

    # Make the prediction. The result is a single number between 0 and 1.
    prediction_score = model.predict(img_array)[0][0]

    # Calculate the confidence for both "Real" and "Fake"
    confidence_real = prediction_score * 100
    confidence_fake = (1 - prediction_score) * 100

    # Determine the final verdict
    final_verdict = "Real" if confidence_real > confidence_fake else "Fake"

    # Print the detailed results
    print(f"Confidence it is FAKE: {confidence_fake:.2f}%")
    print(f"Confidence it is REAL: {confidence_real:.2f}%")
    print(f" Final Verdict: The image appears to be {final_verdict}.")
    print("-" * 50) # Add a separator for clarity


# --- 2. Create the Upload Widget ---
print("Please upload one or more image files to get a detailed prediction.")
uploaded = files.upload()


# --- 3. Loop Through Uploaded Files and Predict ---
if not uploaded:
    print("\nNo files were uploaded. Please run the cell again.")
else:
    print("\n" + "="*50)
    print("                DETAILED PREDICTION RESULTS")
    print("="*50)
    # Loop through each uploaded filename and get its detailed prediction
    for filename in uploaded.keys():
      predict_with_details(filename)

Please upload one or more image files to get a detailed prediction.


In [ ]:
# Evaluate model on validation/test set
loss, acc = model.evaluate(val_generator)
print(f"Validation Accuracy: {acc * 100:.2f}%")

# Generate predictions and true labels
val_generator.reset()
y_pred = model.predict(val_generator)
y_pred_labels = (y_pred > 0.5).astype(int).flatten()

y_true = val_generator.classes

# Class names from directory
class_labels = list(val_generator.class_indices.keys())

# Classification Report
from sklearn.metrics import classification_report
print(classification_report(y_true, y_pred_labels, target_names=class_labels))


In [ ]:
print(classification_report(y_true, y_pred, target_names=['Normal', 'Pneumonia']))